# Geospatial Heatmap Data Visualization

Configure paths and column names:

In [43]:
import os

# paths
SUBPHENOTYPES_DATA_PATH = os.path.abspath(os.path.join("..", "subphenotype", "out", "subphenotypes.csv"))
GENETICS_DATA_PATH      = os.path.abspath(os.path.join("..", "genetic", "data", "state_scores.csv"))
GEOJSON_PATH            = os.path.abspath(os.path.join("..", "subphenotype", "data", "assets", "districts.geojson"))

# column names
PRIORITY_SCORE_COL = "GLP1_Focused_Priority_Score"
GENETIC_RISK_COL   = "Weighted_Priority_Score"
DISTRICT_COL       = "District"
STATE_COL          = "State/UT"

# geojson field names
DISTRICT_GJ  = "district"
STATE_GJ     = "st_nm"

# output path
HTML_PATH    = os.path.abspath(os.path.join(".", "out", "map.html"))

Construct district name alias mapping:

In [44]:
# alias map to reconcile common spelling variants between CSV and GeoJSON
ALIASES = {
    "aizawl":  "aizawal",
    "belgaum": "belagavi",
    "aravali": "aravalli"
}

Construct CSV geo-identifier $\to$ priority score lookup:

In [45]:
import pandas as pd


def norm(s:str) -> str:
    if s is None: return ""
    
    return str(s).strip().lower()


df_subphenotype = pd.read_csv(SUBPHENOTYPES_DATA_PATH)

# build normalized matching keys
df_subphenotype["district_key"] = df_subphenotype[DISTRICT_COL].map(norm).map(lambda x: ALIASES.get(x, x))
df_subphenotype["state_key"]    = df_subphenotype[STATE_COL].map(norm)

# build a lookup from (district_key, state_key) -> score
subphenotype_lookup = { (r["district_key"], r["state_key"]): r[PRIORITY_SCORE_COL] for _, r in df_subphenotype.dropna(subset=[PRIORITY_SCORE_COL]).iterrows() }

Construct genetic data lookup:

In [46]:
df_genetic = pd.read_csv(GENETICS_DATA_PATH)

# build normalized matching keys
df_genetic["state_key"] = df_genetic[STATE_COL].map(norm)

# build a lookup from (district_key, state_key) -> score
gneetic_lookup = { r["state_key"]: r[GENETIC_RISK_COL] for _, r in df_genetic.dropna(subset=[GENETIC_RISK_COL]).iterrows() }

Match subphenotype-based priority and genetic risk scores from CSV, and add to GeoJSON features:

In [47]:
import json

# load .geojson
with open(GEOJSON_PATH, "r", encoding="utf-8") as f:
    geo = json.load(f)

for feat in geo["features"]:
    props = feat.get("properties", {})
    
    d = norm(props.get(DISTRICT_GJ, ""))
    s = norm(props.get(STATE_GJ, ""))

    # match score and risk
    score = subphenotype_lookup.get((d, s))
    risk  = gneetic_lookup.get(s)
    
    props["glp1_score"]   = float(score) if score else None
    props["genetic_risk"] = float(risk) if risk else None
    
    feat["properties"] = props

Draw heatmap:

In [48]:
import folium
from folium.features import GeoJsonTooltip
from branca.colormap import linear
import numpy as np

values     = df_subphenotype[PRIORITY_SCORE_COL].dropna()
VMIN, VMAX = float(values.min()), float(values.max())

risk_values     = df_genetic[GENETIC_RISK_COL].dropna()
RISK_MIN, RISK_MAX = float(risk_values.min()), float(risk_values.max())

# color map palette for GLP1 priority scoring
priority_cmap         = linear.YlOrRd_09.scale(VMIN, VMAX)
priority_cmap.caption = "GLP1-Focused Priority Score"

# color map palette for genetic risk scoring
risk_cmap         = linear.YlOrRd_09.scale(RISK_MIN, RISK_MAX)
risk_cmap.caption = "Genetic Risk Score"


def style_priority(feature):
    val = feature["properties"].get("glp1_score")
    
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return { "fillOpacity": 0.1, "weight": 0.2, "fillColor": "#cccccc", "color": "#555555" }
    
    return {
        "fillOpacity": 0.6,
        "weight":      0.25,
        "fillColor":   priority_cmap(float(val)),
        "color":       "#333333",
    }


def style_risk(feature):
    val = feature["properties"].get("genetic_risk")
    
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return { "fillOpacity": 0.1, "weight": 0.2, "fillColor": "#cccccc", "color": "#555555" }
    
    return {
        "fillOpacity": 0.35,
        "weight":      0.45,
        "fillColor":   risk_cmap(float(val)),
        "color":       "#333333",
    }


# build map
m = folium.Map(location=[22.9734, 78.6569], zoom_start=5, tiles="cartodbpositron")

folium.GeoJson(
    data          =geo,
    name          ="GLP1 Priority by District",
    style_function=style_priority,
    
    tooltip=GeoJsonTooltip(
        fields =[DISTRICT_GJ, STATE_GJ, "glp1_score"],
        aliases=["District", "State/UT", "GLP1 Score"],
        sticky =True,
        labels =True,
    ),
).add_to(m)

folium.GeoJson(
    data          =geo,
    name          ="Genetic Risk by State",
    style_function=style_risk,
    
    tooltip=GeoJsonTooltip(
        fields =[DISTRICT_GJ, STATE_GJ, "genetic_risk"],
        aliases=["District", "State/UT", "Genetic Risk"],
        sticky =True,
        labels =True,
    ),
).add_to(m)

# add the legends
priority_cmap.add_to(m)
risk_cmap.add_to(m)

folium.LayerControl().add_to(m)



Export heatmap to interactive HTML:

In [49]:
OUT_DIR  = os.path.abspath(os.path.join(".", "out"))
OUT_PATH = os.path.join(OUT_DIR, "heatmap.html")

# create if doesn't exist
os.makedirs(OUT_DIR, exist_ok=True)

m.save(OUT_PATH)

print(f"Saved: {OUT_PATH}")

Saved: /home/aarush/Projects/NN_Hackathon/visualization/out/heatmap.html
